<a href="https://colab.research.google.com/github/rahul94jh/MSC-Research/blob/main/clickbait_data_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initial setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
root = '/content/drive/MyDrive/Colab Notebooks/clickbait_detection'

In [ ]:
# cd to root dir
%cd /content/drive/MyDrive/Colab Notebooks/clickbait_detection

/content/drive/MyDrive/Colab Notebooks/clickbait_detection


In [ ]:
!ls

root_dataset  thumbnails


# Manage Libraries

In [ ]:
!pip install jsonlines
!pip install langdetect

In [ ]:
import jsonlines
import random
import numpy as np
import pandas as pd
import requests 
import shutil
import re
import os

from langdetect import detect

# Load data

In [ ]:
data_path = "./root_dataset/groundtruth.json"
data = []
with jsonlines.open(data_path) as reader:
    for obj in reader:
      data.append(obj)

In [ ]:
len(data)

21091

In [ ]:
dict_1 = random.choice(data)
dict_1

{'Comments': ['Everyone has atleast one way to beg for \nlikes\n\n\n\n\n\n\nLike if u agree',
  'Like for hating Josh',
  'I see that swastika...',
  'the nazi symbol for the tire im disappointed in you',
  'omg kwebbelkop is a nazi',
  'THE MACHINE MORE LIKE THE HITLER',
  "I'm Dutch hallo is Ben Nederland ",
  'The weel of jordis car are nazi logo',
  'get in the duck mobile',
  'what about a mech building challenge?',
  'i cant stop laughing when he said guess what i built...a peanus ',
  'swastika!!! Nazi confirmed',
  'history guy sucks dick',
  'you.  are.  cool.    do.  you.   like.  it.',
  'CALL DUTY',
  'ai like the game',
  '',
  'and on this day kwebblekop left alot of subs',
  'The nickname for the spider should be the black widow',
  "it's so nice",
  "josh couldn't get it in...too big",
  "jordy's was the best",
  "Don't hate ",
  'big penis',
  'play more GTA',
  'Scrap mechanic',
  'i love you guys so fucking much',
  'i can tell kwebbelkop might be a. \n\n\n\n\n\nask 

#Helper functions

In [ ]:
def preprocess_string(text):
    text_cleaned = re.sub(r"http\S+", "", text, flags=re.IGNORECASE|re.MULTILINE)  # remove any url in the text
    text_cleaned = re.sub(r"\w+@\w{1,}\.\w{1,}","", text_cleaned, flags=re.IGNORECASE|re.MULTILINE) # remove emails from text
    text_cleaned = re.sub(r'[^a-z\s]+',' ',text_cleaned,flags=re.IGNORECASE|re.MULTILINE) # remove non alphabetics char
    text_cleaned = re.sub(r'(\s+)',' ',text_cleaned, flags=re.IGNORECASE|re.MULTILINE)  # remove spaces

    text_cleaned = text_cleaned.lstrip()
    text_cleaned = text_cleaned.rstrip()
    text_cleaned = text_cleaned.lower() 

    return text_cleaned 

def extract_textual_column(row, column_name):
  snippet_dict = row
  text = row[column_name]
  cleaned_text = preprocess_string(text)
  return cleaned_text

def extract_thumbnail_url(row, column_name, sub_column):
  snippet = row
  url = row[column_name][sub_column]['url']
  return url

def extract_language_info(row, column):
  if type(row) is list :
    return row[0][column]
  else:
    return ''

def extract_from_dict(row, column):
  return row[column]

def detect_language(row):
  lang = row['language']
  if lang is '':
    return detect(row['description'])
  else:
    return lang

def derive_comments(row):
  if len(row['comments']) > 0:
    return row['comments']
  elif len(row['Comments']) > 0 :
    return row['Comments']
  else:
    return ''

# Pre processing

In [ ]:
df = pd.DataFrame.from_dict(data)
df = df.drop('_id',axis=1)
df.set_index('id',inplace=True)
df.head(1)

,statistics,snippet,comments,kind,relatedVideos,etag,misleadingComments,baitComments,thumbnailComments,clickbaitComments,deceptiveComments,deceivingComments,clickedComments,flaggedComments,titleComments,detectedLanguage,clickbaitClassification,usedForTraining,Comments
id,,,,,,,,,,,,,,,,,,,
W2WgTE9OKyg,"{'favoriteCount': '0', 'commentCount': '9777',...","{'liveBroadcastContent': 'none', 'description'...","[5 seconds in the video already cringing, The ...",youtube#video,"[2J-mcVT1PH4, eWK5DUlaJhw, AWTEZ92Wi2Q, 7shkSN...","""I_8xdZu766_FSaexEaDXTIfEWc0/9s_iSeNX2g-Iu2k45...","[Reported for misleading thumbnail., Misleadin...","[Where's the naked chick?, How did this unfunn...",[Came for the clickbait\nStayed for Cactus Man...,[Came for the clickbait\nStayed for Cactus Man...,[I'm sure most of your thumbs down will be bec...,[Everyone flag the video for deceiving thumbna...,[The girl on the thumbnail is *Melisa Mendiny*...,"[Flagged for misleading content., Flagged for ...","[this waws bad :(\n\n(i wanted boobie girl), Y...","[{'confidence': 9.75, 'isReliable': True, 'lan...","{'clickbait': 'yes', 'comments': 'massive clic...",1,NaN


In [ ]:
df_filtered = df.loc[:,['snippet','comments','detectedLanguage','clickbaitClassification','Comments', 'usedForTraining']]
df_filtered.head(2)

,snippet,comments,detectedLanguage,clickbaitClassification,Comments,usedForTraining
id,,,,,,
W2WgTE9OKyg,"{'liveBroadcastContent': 'none', 'description'...","[5 seconds in the video already cringing, The ...","[{'confidence': 9.75, 'isReliable': True, 'lan...","{'clickbait': 'yes', 'comments': 'massive clic...",NaN,1
eWK5DUlaJhw,{'title': 'EL VIDEO MAS VISTO DE YOUTUBE 2016'...,[Todos sabemos que la cantidad de visitas fuer...,"[{'confidence': 9.21, 'isReliable': True, 'lan...","{'clickbait': 'default', 'comments': ''}",NaN,NaN


In [ ]:
df_filtered['title'] = df_filtered.loc[:,'snippet'].apply(lambda x : extract_textual_column(x,'title'))

In [ ]:
df_filtered['description'] = df_filtered.loc[:,'snippet'].apply(lambda x : extract_textual_column(x,'description'))

In [ ]:
df_filtered['thumbnail_320_180'] = df_filtered.loc[:,'snippet'].apply(lambda x : extract_thumbnail_url(x,'thumbnails', 'medium'))

In [ ]:
df_filtered['language'] = df_filtered.loc[:,'detectedLanguage'].apply(lambda x : extract_language_info(x,'language'))

In [ ]:
df_filtered['language_reliable'] = df_filtered.loc[:,'detectedLanguage'].apply(lambda x : extract_language_info(x,'isReliable'))

In [ ]:
df_filtered['is_clickbait'] = df_filtered.loc[:,'clickbaitClassification'].apply(lambda x : extract_from_dict(x,'clickbait'))

In [ ]:
df_filtered['clickbait_comments'] = df_filtered.loc[:,'clickbaitClassification'].apply(lambda x : extract_from_dict(x,'comments'))

In [ ]:
df_filtered = df_filtered.drop(['snippet','detectedLanguage','clickbaitClassification'], axis=1)

In [ ]:
df_filtered.loc[((df_filtered.comments.isnull()) & (~df_filtered.Comments.isnull()))]

,comments,Comments,usedForTraining,title,description,thumbnail_320_180,language,language_reliable,is_clickbait,clickbait_comments
id,,,,,,,,,,
2YNx-wFdiEw,NaN,"[Brazil just got me I died of laughing, 8:25 I...",1,hot woman anchor funny news bloopers fails apr...,lets smash likes for more funny epic news bloo...,https://i.ytimg.com/vi/2YNx-wFdiEw/mqdefault.jpg,en,True,yes-sampling,
T3djXcx2ewQ,NaN,[Wow I miss this show. Seeing all those celebr...,1,stupid game show answers you fool,from a episode of hollywood squares this is on...,https://i.ytimg.com/vi/T3djXcx2ewQ/mqdefault.jpg,en,True,no-sampling,
g6dLefwa0Vc,NaN,"[, Funny VideoS 2016 , funny,videos,2016,dog a...",1,best news bloopers fail hd,best news bloopers january best news bloopers ...,https://i.ytimg.com/vi/g6dLefwa0Vc/mqdefault.jpg,en,True,default-sampling,
FspuwbI3fKM,NaN,"[That train - I was at Princeton, NJ, station ...",1,funniest news bloopers april,funny videos news bloopers laugh smile or grin...,https://i.ytimg.com/vi/FspuwbI3fKM/mqdefault.jpg,en,True,no-sampling,
e_gWsWcs3Qc,NaN,[That guy at the campfire was a great improvis...,1,conan becomes a civil war reenactor conan on tbs,finally something that both southern rebels an...,https://i.ytimg.com/vi/e_gWsWcs3Qc/mqdefault.jpg,en,True,no-sampling,
...,...,...,...,...,...,...,...,...,...,...
OJelJNzcLGw,NaN,[],NaN,sneak peek the omnivore s dilemma by michael p...,thnkr gives you a taste of what tom colicchio ...,https://i.ytimg.com/vi/OJelJNzcLGw/mqdefault.jpg,,,no,channels
75jc1fR7_qs,NaN,[This one's a gem. I come back every so often ...,NaN,jason silva s epiphany addiction,meet jason silva futurist filmmaker and idea d...,https://i.ytimg.com/vi/75jc1fR7_qs/mqdefault.jpg,,,no,channels
ff7FKsObhag,NaN,[everything is possible except the time travel...,NaN,yr old wunderkind sneak peek tanishq abraham,at only tanishq abraham is one of america s yo...,https://i.ytimg.com/vi/ff7FKsObhag/mqdefault.jpg,,,no,channels


In [ ]:
df_filtered.loc[(df_filtered.comments.isnull()),'comments'] = ''

In [ ]:
df_filtered.loc[(df_filtered.Comments.isnull()),'Comments'] = ''

In [ ]:
df_filtered['all_comments'] = df_filtered.apply(lambda x : derive_comments(x), axis=1)

In [ ]:
df_filtered.loc[(df_filtered.all_comments == '')].shape

(485, 11)

In [ ]:
df_filtered = df_filtered.drop(['comments','Comments'], axis=1)

In [ ]:
df_filtered.head(2)

,usedForTraining,title,description,thumbnail_320_180,language,language_reliable,is_clickbait,clickbait_comments,all_comments
id,,,,,,,,,
W2WgTE9OKyg,1,very funny videos hd,facebook,https://i.ytimg.com/vi/W2WgTE9OKyg/mqdefault.jpg,en,True,yes,massive clickbait,"[5 seconds in the video already cringing, The ..."
eWK5DUlaJhw,NaN,el video mas visto de youtube,este es mi otro canal el principal donde subo ...,https://i.ytimg.com/vi/eWK5DUlaJhw/mqdefault.jpg,es,True,default,,[Todos sabemos que la cantidad de visitas fuer...


# Analyze and process

In [ ]:
df_filtered.loc[(df_filtered.title.isnull())].shape

(0, 9)

In [ ]:
df_filtered.loc[(df_filtered.description.isnull())].shape

(0, 9)

In [ ]:
print(df_filtered.loc[(df_filtered.description=='')].shape)
print(df_filtered.loc[(df_filtered.title=='')].shape)

(164, 9)
(7, 9)


In [ ]:
df_filtered = df_filtered.loc[~((df_filtered.title=='')| (df_filtered.description==''))]

In [ ]:
df_filtered.language.value_counts()

           17459
en          3432
hi             9
ru             6
th             3
ja             3
ar             2
bh             2
uk             1
ko             1
es             1
zh-Hant        1
bn             1
Name: language, dtype: int64

In [ ]:
df_filtered.language_reliable.value_counts()

         17459
True      3169
False      293
Name: language_reliable, dtype: int64

In [ ]:
df_filtered.loc[(df_filtered.language_reliable == ''),'language_reliable'] = False

In [ ]:
df_filtered['language'] = df_filtered.apply(lambda x : detect_language(x), axis=1)

In [ ]:
df_filtered = df_filtered.loc[(df_filtered.language.isin(['en']))]

In [ ]:
df_filtered.head(1)

,usedForTraining,title,description,thumbnail_320_180,language,language_reliable,is_clickbait,clickbait_comments,all_comments
id,,,,,,,,,
W2WgTE9OKyg,1,very funny videos hd,facebook,https://i.ytimg.com/vi/W2WgTE9OKyg/mqdefault.jpg,en,True,yes,massive clickbait,"[5 seconds in the video already cringing, The ..."


In [ ]:
df_filtered.loc[~(df_filtered.usedForTraining.isin(['1'])),'usedForTraining'] = '0'

In [ ]:
df_filtered['usedForTraining'] = df_filtered['usedForTraining'].astype('int')
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20847 entries, W2WgTE9OKyg to mdjmjWlR-po
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   usedForTraining     20847 non-null  int64 
 1   title               20847 non-null  object
 2   description         20847 non-null  object
 3   thumbnail_320_180   20847 non-null  object
 4   language            20847 non-null  object
 5   language_reliable   20847 non-null  object
 6   is_clickbait        20847 non-null  object
 7   clickbait_comments  20847 non-null  object
 8   all_comments        20847 non-null  object
dtypes: int64(1), object(8)
memory usage: 1.6+ MB


In [ ]:
df_filtered.is_clickbait.value_counts()

yes                 10004
no                   9372
no-sampling           622
default               408
yes-sampling          327
default-sampling      114
Name: is_clickbait, dtype: int64

In [ ]:
df_filtered = df_filtered.loc[(df_filtered.is_clickbait.isin(['yes','no']))]

In [ ]:
df_filtered.loc[((df_filtered.clickbait_comments !='') & (df_filtered.clickbait_comments !='channels'))]

,usedForTraining,title,description,thumbnail_320_180,language,language_reliable,is_clickbait,clickbait_comments,all_comments
id,,,,,,,,,
W2WgTE9OKyg,1,very funny videos hd,facebook,https://i.ytimg.com/vi/W2WgTE9OKyg/mqdefault.jpg,en,True,yes,massive clickbait,"[5 seconds in the video already cringing, The ..."
AWTEZ92Wi2Q,1,crazy funny videos youtube,crazy funny videos youtube watch more funny vi...,https://i.ytimg.com/vi/AWTEZ92Wi2Q/mqdefault.jpg,en,True,yes,thumbnail not in video,[Crazy funny videos youtube\n\nThanks for watc...
7shkSNjmMus,1,new funny videos pranks part try not to laugh ...,the best funny videos and funny fails compilat...,https://i.ytimg.com/vi/7shkSNjmMus/mqdefault.jpg,en,True,yes,misleading thumbnail,"[great video !!\nbig like !, i hate when minia..."
ydDvZ0jDcCg,1,funny videos try not to laugh impossible chall...,funny videos try not to laugh impossible chall...,https://i.ytimg.com/vi/ydDvZ0jDcCg/mqdefault.jpg,en,True,yes,misleading thumbnail,[*Watch more funny video here : *\n*https://ww...
8amLdrMTEJE,1,funniest fight ever,a fight of a fat guy vs a thin guy,https://i.ytimg.com/vi/8amLdrMTEJE/mqdefault.jpg,en,True,yes,somewhat misleading title,"[This is why weight divisions were made., Agar..."
...,...,...,...,...,...,...,...,...,...
uCXKSb2HGWQ,1,used condom prank on boyfriend gone wrong,whats good d b nation its ya boy damien biannc...,https://i.ytimg.com/vi/uCXKSb2HGWQ/mqdefault.jpg,en,True,yes,Thumbnail,[flagged video for misleading thumbnail...i fu...
m8zf7kJOQ0M,1,epic scare prank extra scary pranks of may top...,watch the latest and scariest funny pranks in ...,https://i.ytimg.com/vi/m8zf7kJOQ0M/mqdefault.jpg,en,True,yes,Thumbnail not in video,"[I like this time 6:51 ha ha ha, song at 10.16..."
yoYZB1wf17c,1,worst plane crashes,please like subscribe if you enjoyed the video...,https://i.ytimg.com/vi/yoYZB1wf17c/mqdefault.jpg,en,False,yes,Thumbnail,"[*Heart failure, strokes, blood clots, and acc..."


In [ ]:
df_filtered.is_clickbait = df_filtered.is_clickbait.astype('category')
df_filtered.is_clickbait = df_filtered.is_clickbait.cat.codes
df_filtered.head()

,usedForTraining,title,description,thumbnail_320_180,language,language_reliable,is_clickbait,clickbait_comments,all_comments
id,,,,,,,,,
W2WgTE9OKyg,1,very funny videos hd,facebook,https://i.ytimg.com/vi/W2WgTE9OKyg/mqdefault.jpg,en,True,1,massive clickbait,"[5 seconds in the video already cringing, The ..."
AWTEZ92Wi2Q,1,crazy funny videos youtube,crazy funny videos youtube watch more funny vi...,https://i.ytimg.com/vi/AWTEZ92Wi2Q/mqdefault.jpg,en,True,1,thumbnail not in video,[Crazy funny videos youtube\n\nThanks for watc...
7shkSNjmMus,1,new funny videos pranks part try not to laugh ...,the best funny videos and funny fails compilat...,https://i.ytimg.com/vi/7shkSNjmMus/mqdefault.jpg,en,True,1,misleading thumbnail,"[great video !!\nbig like !, i hate when minia..."
9CezzUlt56s,1,whatsapp best funny video whatsapp best video ...,here are e new latest viral whatsapp indian fu...,https://i.ytimg.com/vi/9CezzUlt56s/mqdefault.jpg,en,True,0,,"[hey my sweet friends thank you everyone, Than..."
ydDvZ0jDcCg,1,funny videos try not to laugh impossible chall...,funny videos try not to laugh impossible chall...,https://i.ytimg.com/vi/ydDvZ0jDcCg/mqdefault.jpg,en,True,1,misleading thumbnail,[*Watch more funny video here : *\n*https://ww...


In [ ]:
#df_filtered['thumbnail_320_180_img']=''

# Image mapping

## Setup

In [ ]:
clickbaitImage_dir = '/content/drive/MyDrive/Colab Notebooks/clickbait_detection/thumbnails/clickbaits'
nonclickbaitsImage_dir = '/content/drive/MyDrive/Colab Notebooks/clickbait_detection/thumbnails/nonclickbaits'

if not os.path.exists(clickbaitImage_dir):
    os.makedirs(clickbaitImage_dir)

if not os.path.exists(nonclickbaitsImage_dir):
    os.makedirs(nonclickbaitsImage_dir)

In [ ]:
def download_save_image(df):
  df_new = df.copy(deep=True)

  for i in range(len(df_new)):
    row = df_new.iloc[i]

    id = row['id']  
    url = row['thumbnail_320_180'] 
    is_clickbait = row['is_clickbait'] 


    filename = f'{id}' + '_' + url.split("/")[-1]      

    if is_clickbait:      
        image_path = os.path.join(clickbaitImage_dir,filename) 
    else:
        image_path = os.path.join(nonclickbaitsImage_dir,filename) 

     
    if not os.path.exists(image_path):                       
            r = requests.get(url, stream = True)             
            
            if r.status_code == 200:                         
                r.raw.decode_content = True

                with open(image_path,'wb') as f:
                    shutil.copyfileobj(r.raw, f)             

                print(i, ': success')

                df_new.loc[(df_new['id'] == id), 'thumbnail_320_180_img'] = image_path        # replace url with image path  

            else:
                print(i, ': failed')

                df_new.loc[(df_new['id'] == id), 'thumbnail_320_180_img'] = 'NA'                 # replace url with empty string 

    else:
        df_new.loc[(df_new['id'] == id), 'thumbnail_320_180_img'] = image_path 
        print('file exist', image_path)

  return df_new 

In [ ]:
def update_dataframe(df_p, df_c, index_col='id'):
  df_1 = df_p.copy()
  df_2 = df_c.copy()
  df_1.set_index(index_col, inplace=True)
  df_1.update(df_2.set_index(index_col))
  df_1 = df_1.reset_index() 
  return df_1

In [ ]:
df_filtered = df_filtered.reset_index()

## Download

In [ ]:
save_data_path = "/content/drive/MyDrive/Colab Notebooks/clickbait_detection/root_dataset/clickbait_with_image.csv"
#if os.path.exists(save_data_path):
    #df_filtered = pd.read_csv(save_data_path)


In [ ]:
# Pick random samples
sampled_df = df_filtered.loc[((df_filtered['thumbnail_320_180_img'].isin([''])) | (df_filtered['thumbnail_320_180_img'].isnull()) )].sample(2000)

In [ ]:
df_filtered = download_save_image(df_filtered)

Streaming output truncated to the last 5000 lines.
14376 : success
file exist /content/drive/MyDrive/Colab Notebooks/clickbait_detection/thumbnails/nonclickbaits/9zso7ChaQXQ_mqdefault.jpg
file exist /content/drive/MyDrive/Colab Notebooks/clickbait_detection/thumbnails/nonclickbaits/TQxeutcYP6I_mqdefault.jpg
14379 : success
14380 : success
14381 : success
file exist /content/drive/MyDrive/Colab Notebooks/clickbait_detection/thumbnails/nonclickbaits/liqF6EamiE4_mqdefault.jpg
file exist /content/drive/MyDrive/Colab Notebooks/clickbait_detection/thumbnails/nonclickbaits/YJbKieEC49M_mqdefault.jpg
14384 : success
14385 : success
14386 : success
14387 : success
14388 : success
14389 : success
file exist /content/drive/MyDrive/Colab Notebooks/clickbait_detection/thumbnails/nonclickbaits/lYc7kHXW0xg_mqdefault.jpg
file exist /content/drive/MyDrive/Colab Notebooks/clickbait_detection/thumbnails/nonclickbaits/zuMjnP1YbPw_mqdefault.jpg
14392 : success
14393 : success
14394 : success
14395 : success

In [ ]:
# filter out rows where image download was unsuccessful 
df_filtered =  df_filtered.loc[(df_filtered['thumbnail_320_180_img'] != 'NA')]

In [ ]:
df_filtered.shape

(15348, 11)

In [ ]:
# save the dataframe
df_filtered.to_csv(save_data_path, index=False)

In [ ]:
df_filtered